In [3]:
import math
import ecdsa
import ecdsa.ellipticcurve as EC

def inv_mod_p(x, p):
    if 1 != math.gcd(x, p):
        raise ValueError("Arguments not prime")
    q11 = 1
    q22 = 1
    q12 = 0
    q21 = 0
    while p != 0:
        temp = p
        q = x // p
        p = x % p
        x = temp
        t21 = q21
        t22 = q22
        q21 = q11 - q*q21
        q22 = q12 - q*q22
        q11 = t21
        q12 = t22
    return q11

curve = ecdsa.SECP256k1
G = curve.generator
n = G.order()

## point represented by the public key
## 0x0429a272134c84cd93c1520a85007c2a0e38e80d0f312266b88089c17b61c48101b0257964abfb5640adc754419df69b142f773cb3687fe6a1601b3f07c71904b2
x = int('29a272134c84cd93c1520a85007c2a0e38e80d0f312266b88089c17b61c48101', 16)
y = int('b0257964abfb5640adc754419df69b142f773cb3687fe6a1601b3f07c71904b2', 16)
Q = EC.Point(curve.curve, x, y)
pubkey = ecdsa.VerifyingKey.from_public_point(Q, curve)

# get a random x oordinate within the order of the curve
a = ecdsa.util.randrange(n-1)

# now see if a random b has a relationship with the a
valid_s = False
while not valid_s:
    b = ecdsa.util.randrange(n-1)
    b_inv = inv_mod_p(b, n)
    
    K = (a*G) + (b*Q)  ##"recover" a potential point
    r = K.x() % n      ##the x ordinate of the point 
    
    s = r * b_inv % n  ##the "s" of the signature

    if 0 < s < n:
        valid_s = True ##the "s" derived from r is good

m = (((a * r) % n) * b_inv) % n   #message hash implied by the r

message_bytes32 = format(m, '064x')
r_bytes32 = format(r, '064x')
s_bytes32 = format(s, '064x')

print("message: " + message_bytes32)
print("r: " + r_bytes32)
print("s: " + s_bytes32)

sig = ecdsa.ecdsa.Signature(r, s)
if pubkey.pubkey.verifies(m, sig):
    print("SIGNATURE VERIFIED")
else:
    print("FAILED TO VERIFY")

1
message: b6d8d05c2b97d08927dbdb647e8a15f91846104ad1d34be5fa138cb7e3cbde06
r: 2ad5e3fdc18939c174cfb07e0af9b246c6224318b06093699d156dd59d4ecdec
s: 71f72ba5934bf833477bd07036999e4335d10be47721b15d6f560388a91debe9
SIGNATURE VERIFIED


In [2]:
len('0429a272134c84cd93c1520a85007c2a0e38e80d0f312266b88089c17b61c48101b0257964abfb5640adc754419df69b142f773cb3687fe6a1601b3f07c71904b2')

130